In [4]:
import sklearn
import xgboost
import pkgutil
import os
import importlib, inspect
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
import pickle

save_dir = "./models/sklearn/{0}".format(sklearn.__version__)
# print(save_dir)
os.mkdir(save_dir)

# Generate scikit-learn model samples for rocket

The script will search for all the classifiers and regressors using "ClassifierMixin" and "RegressorMixin" as the first condition, and "fit" as the second condition in scikit-learn's package (`sklearn`)

In [5]:
def get_algorithms(package, prefix="", output_file="sklearn_algorithms.txt"):
    algorithms = {}
    algorithms["classification"] = []
    algorithms["regression"] = []
    
    fout = open(output_file, "w+")
    number_of_regressors = 0
    number_of_classifiers = 0

    for module in package.__all__:
        try:
            modname = prefix + module
            mod = importlib.import_module(modname)

            for name, obj in inspect.getmembers(mod):
                if inspect.isclass(obj) and (issubclass(obj, ClassifierMixin)):
                    # test = importlib.import_module(obj)
                    if "fit" in dir(obj): # only use those that got fit function
                        algorithms["classification"].append(obj)
                        fout.write("classification,{0}\n".format(str(name)))
                        number_of_classifiers = number_of_classifiers + 1

                elif inspect.isclass(obj) and issubclass(obj, RegressorMixin):
                    # test = importlib.import_module(obj)
                    if "fit" in dir(obj): # only use those that got fit function
                        algorithms["regression"].append(obj)
                        fout.write("regression,{0}\n".format(str(name)))
                        number_of_regressors = number_of_regressors + 1
                        
        except ModuleNotFoundError as e:
            continue

    fout.write("=================================\n")
    fout.write("Total number of classifiers: {0}\n".format(number_of_classifiers))
    fout.write("Total number of regerssors: {0}\n".format(number_of_regressors))
    return algorithms

prefix = "sklearn."
from sklearn.base import ClassifierMixin, RegressorMixin
algorithms = get_algorithms(sklearn, prefix)

In [10]:
def train_models(algorithms, model_type, dataset):
    """This method will mass train models given in the algorithms.
    
    Args:
        algorithms (list of objects)
        model_type (str) use to prefix log files
        dataset (str) use to prefix log files
    """
    # will overwrite existing files
    # tracking purpose
    stats = open("{0}_stats.log".format(model_type), "w+")
    error = open("{0}_error.log".format(model_type), "w+")

    for algorithm in algorithms:
        try:
            uniquename = algorithm.__module__, algorithm.__name__

            if "estimator" in inspect.signature(algorithm).parameters.keys():
                clf = algorithm(LinearSVC(random_state=42))
            else:
                clf = algorithm()

            print("scoring {0}".format(uniquename))
            score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)

            # statistics for internal refernece
            stats.write("{0},{1}\n".format(uniquename, score))
            # write to pickle model
            with open("algorithm.txt", "a+") as f:
                f.write("{0}.{1}\n".format(algorithm.__module__, algorithm.__name__))
            pickle.dump(clf, open("{4}/{0}_{1}_{2}.{3}.sav".format(model_type, dataset, algorithm.__module__, algorithm.__name__, save_dir), "wb+"))
        except:
            # some we can't do will skip, but we keep track first. might need special parameters to run them
            print("skipping",  algorithm.__module__)
            error.write("skipping {0}\n".format(algorithm.__module__))
            
            import traceback
            traceback.print_exc()
            continue

## Binary Classification Models

In [11]:
compas_training = pickle.load(open("./data/pickle_pandas_tabular_compas_training.sav", "rb"))
compas_testing = pickle.load(open("./data/pickle_pandas_tabular_compas_testing.sav", "rb"))

# Consider to do more tests
# cardio_testing = pickle.load(open("./data/pickle_pandas_tabular_cardio_testing.sav", "rb"))
# cardio_training = pickle.load(open("./data/pickle_pandas_tabular_cardio_training.sav", "rb"))

X_train = compas_training.drop("two_year_recid", axis=1)
y_train = compas_training[["two_year_recid"]]
X_test = compas_testing.drop("two_year_recid", axis=1)
y_test = compas_testing[["two_year_recid"]]

classifiers = algorithms["classification"]
train_models(classifiers, "binary_classification", "compas")

scoring ('sklearn.calibration', 'CalibratedClassifierCV')
scoring ('sklearn.svm._classes', 'LinearSVC')
scoring ('sklearn.dummy', 'DummyClassifier')
scoring ('sklearn.ensemble._weight_boosting', 'AdaBoostClassifier')
skipping sklearn.ensemble._weight_boosting
scoring ('sklearn.ensemble._bagging', 'BaggingClassifier')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py", line 142, in fit
    self._validate_estimator()
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py", line 517, in _validate_estimator
    raise TypeError(
TypeError: AdaBoostClassifier with algorithm='SAMME.R' requires that the weak learner supports the calculation of class probabilities with a predict_proba method.
Please change the base estimator or set algorithm='SAMME' instead.


scoring ('sklearn.ensemble._forest', 'ExtraTreesClassifier')
scoring ('sklearn.ensemble._gb', 'GradientBoostingClassifier')
scoring ('sklearn.ensemble._hist_gradient_boosting.gradient_boosting', 'HistGradientBoostingClassifier')
scoring ('sklearn.ensemble._forest', 'RandomForestClassifier')
skipping sklearn.ensemble._stacking
skipping sklearn.ensemble._voting
scoring ('sklearn.gaussian_process._gpc', 'GaussianProcessClassifier')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: VotingClassifier.__init__() missing 1 required positional argument: 'estimators'


scoring ('sklearn.linear_model._logistic', 'LogisticRegression')
scoring ('sklearn.linear_model._logistic', 'LogisticRegressionCV')
scoring ('sklearn.linear_model._passive_aggressive', 'PassiveAggressiveClassifier')
scoring ('sklearn.linear_model._perceptron', 'Perceptron')
scoring ('sklearn.linear_model._ridge', 'RidgeClassifier')
scoring ('sklearn.linear_model._ridge', 'RidgeClassifierCV')
scoring ('sklearn.linear_model._stochastic_gradient', 'SGDClassifier')
scoring ('sklearn.multiclass', 'OneVsOneClassifier')
scoring ('sklearn.multiclass', 'OneVsRestClassifier')
scoring ('sklearn.multiclass', 'OutputCodeClassifier')
skipping sklearn.multioutput
scoring ('sklearn.multioutput', 'MultiOutputClassifier')
skipping sklearn.multioutput
scoring ('sklearn.naive_bayes', 'BernoulliNB')
scoring ('sklearn.naive_bayes', 'CategoricalNB')
scoring ('sklearn.naive_bayes', 'ComplementNB')
scoring ('sklearn.naive_bayes', 'GaussianNB')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: _BaseChain.__init__() missing 1 required positional argument: 'base_estimator'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
TypeError: MultiOutputClassifier.fit() missing 1 required positional argument: 'Y'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: Can't instantiate abstract class _BaseDiscreteNB with abstract methods _count, _joint_log_likelihood, _update_feature_log_prob
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/base.py", line 668, i

scoring ('sklearn.naive_bayes', 'MultinomialNB')
skipping sklearn.naive_bayes
scoring ('sklearn.neighbors._classification', 'KNeighborsClassifier')
scoring ('sklearn.neighbors._nearest_centroid', 'NearestCentroid')
scoring ('sklearn.neighbors._classification', 'RadiusNeighborsClassifier')
skipping sklearn.neighbors._classification
scoring ('sklearn.neural_network._multilayer_perceptron', 'MLPClassifier')
scoring ('sklearn.semi_supervised._label_propagation', 'LabelPropagation')
scoring ('sklearn.semi_supervised._label_propagation', 'LabelSpreading')
scoring ('sklearn.svm._classes', 'LinearSVC')
scoring ('sklearn.svm._classes', 'NuSVC')
scoring ('sklearn.svm._classes', 'SVC')
scoring ('sklearn.tree._classes', 'DecisionTreeClassifier')
scoring ('sklearn.tree._classes', 'ExtraTreeClassifier')
scoring ('sklearn.discriminant_analysis', 'LinearDiscriminantAnalysis')
scoring ('sklearn.discriminant_analysis', 'QuadraticDiscriminantAnalysis')


## Multiclass Classification Models

In [12]:
loan_training = pickle.load(open("./data/pickle_pandas_tabular_loan_training.sav", "rb")).sample(5000)
loan_testing = pickle.load(open("./data/pickle_pandas_tabular_loan_testing.sav", "rb")).sample(2000)

X_train = loan_training.drop("Interest_Rate", axis=1)
y_train = loan_training[["Interest_Rate"]]
X_test = loan_testing.drop("Interest_Rate", axis=1)
y_test = loan_testing[["Interest_Rate"]]

loan_training.describe()

classifiers = algorithms["classification"]
train_models(classifiers, "multiclass_classification", "loan")

scoring ('sklearn.calibration', 'CalibratedClassifierCV')
scoring ('sklearn.svm._classes', 'LinearSVC')
scoring ('sklearn.dummy', 'DummyClassifier')
scoring ('sklearn.ensemble._weight_boosting', 'AdaBoostClassifier')
skipping sklearn.ensemble._weight_boosting
scoring ('sklearn.ensemble._bagging', 'BaggingClassifier')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py", line 142, in fit
    self._validate_estimator()
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py", line 517, in _validate_estimator
    raise TypeError(
TypeError: AdaBoostClassifier with algorithm='SAMME.R' requires that the weak learner supports the calculation of class probabilities with a predict_proba method.
Please change the base estimator or set algorithm='SAMME' instead.


scoring ('sklearn.ensemble._forest', 'ExtraTreesClassifier')
scoring ('sklearn.ensemble._gb', 'GradientBoostingClassifier')
scoring ('sklearn.ensemble._hist_gradient_boosting.gradient_boosting', 'HistGradientBoostingClassifier')
scoring ('sklearn.ensemble._forest', 'RandomForestClassifier')
skipping sklearn.ensemble._stacking
skipping sklearn.ensemble._voting
scoring ('sklearn.gaussian_process._gpc', 'GaussianProcessClassifier')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: VotingClassifier.__init__() missing 1 required positional argument: 'estimators'


scoring ('sklearn.linear_model._logistic', 'LogisticRegression')
scoring ('sklearn.linear_model._logistic', 'LogisticRegressionCV')
scoring ('sklearn.linear_model._passive_aggressive', 'PassiveAggressiveClassifier')
scoring ('sklearn.linear_model._perceptron', 'Perceptron')
scoring ('sklearn.linear_model._ridge', 'RidgeClassifier')
scoring ('sklearn.linear_model._ridge', 'RidgeClassifierCV')
scoring ('sklearn.linear_model._stochastic_gradient', 'SGDClassifier')
scoring ('sklearn.multiclass', 'OneVsOneClassifier')
scoring ('sklearn.multiclass', 'OneVsRestClassifier')
scoring ('sklearn.multiclass', 'OutputCodeClassifier')
skipping sklearn.multioutput
scoring ('sklearn.multioutput', 'MultiOutputClassifier')
skipping sklearn.multioutput
scoring ('sklearn.naive_bayes', 'BernoulliNB')
scoring ('sklearn.naive_bayes', 'CategoricalNB')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: _BaseChain.__init__() missing 1 required positional argument: 'base_estimator'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
TypeError: MultiOutputClassifier.fit() missing 1 required positional argument: 'Y'


scoring ('sklearn.naive_bayes', 'ComplementNB')
scoring ('sklearn.naive_bayes', 'GaussianNB')
scoring ('sklearn.naive_bayes', 'MultinomialNB')
skipping sklearn.naive_bayes
scoring ('sklearn.neighbors._classification', 'KNeighborsClassifier')
scoring ('sklearn.neighbors._nearest_centroid', 'NearestCentroid')
scoring ('sklearn.neighbors._classification', 'RadiusNeighborsClassifier')
skipping sklearn.neighbors._classification
scoring ('sklearn.neural_network._multilayer_perceptron', 'MLPClassifier')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: Can't instantiate abstract class _BaseDiscreteNB with abstract methods _count, _joint_log_likelihood, _update_feature_log_prob
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/base.py", line 668, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/neighbors/_classification.py", line 597, in predict
    probs = self.predict_proba(X)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/neighbors/_classification.py", line 656, in predict_proba
    raise ValueError(
ValueError: No neighbors found for test samples 

scoring ('sklearn.semi_supervised._label_propagation', 'LabelPropagation')
scoring ('sklearn.semi_supervised._label_propagation', 'LabelSpreading')
scoring ('sklearn.svm._classes', 'LinearSVC')
scoring ('sklearn.svm._classes', 'NuSVC')
scoring ('sklearn.svm._classes', 'SVC')
scoring ('sklearn.tree._classes', 'DecisionTreeClassifier')
scoring ('sklearn.tree._classes', 'ExtraTreeClassifier')
scoring ('sklearn.discriminant_analysis', 'LinearDiscriminantAnalysis')
scoring ('sklearn.discriminant_analysis', 'QuadraticDiscriminantAnalysis')


## Regression Models

In [13]:
insurance_training = pickle.load(open("./data/pickle_pandas_tabular_insurance_training.sav", "rb"))
insurance_testing = pickle.load(open("./data/pickle_pandas_tabular_insurance_testing.sav", "rb"))
insurance_testing.describe()

X_train = insurance_training.drop("charges", axis=1)
y_train = insurance_training[["charges"]]
X_test = insurance_testing.drop("charges", axis=1)
y_test = insurance_testing[["charges"]]

regressors = algorithms["regression"]
train_models(regressors, "regression", "insurance")

scoring ('sklearn.isotonic', 'IsotonicRegression')
skipping sklearn.isotonic
scoring ('sklearn.calibration', '_SigmoidCalibration')
skipping sklearn.calibration
scoring ('sklearn.cross_decomposition._pls', 'CCA')
skipping sklearn.cross_decomposition._pls
scoring ('sklearn.cross_decomposition._pls', 'PLSCanonical')
skipping sklearn.cross_decomposition._pls
scoring ('sklearn.cross_decomposition._pls', 'PLSRegression')
skipping sklearn.cross_decomposition._pls
scoring ('sklearn.dummy', 'DummyRegressor')
scoring ('sklearn.ensemble._weight_boosting', 'AdaBoostRegressor')
skipping sklearn.ensemble._weight_boosting
scoring ('sklearn.ensemble._bagging', 'BaggingRegressor')
skipping sklearn.ensemble._bagging
scoring ('sklearn.ensemble._forest', 'ExtraTreesRegressor')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/isotonic.py", line 351, in fit
    X, y = self._build_y(X, y, sample_weight)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/isotonic.py", line 266, in _build_y
    self._check_input_data_shape(X)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/isotonic.py", line 250, in _check_input_data_shape
    raise ValueError(msg)
ValueError: Isotonic regression input X should be a 1d array or 2d array with 1 feature
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/calibration.py",

scoring ('sklearn.ensemble._gb', 'GradientBoostingRegressor')
scoring ('sklearn.ensemble._hist_gradient_boosting.gradient_boosting', 'HistGradientBoostingRegressor')
scoring ('sklearn.ensemble._forest', 'RandomForestRegressor')
skipping sklearn.ensemble._stacking
skipping sklearn.ensemble._voting
scoring ('sklearn.gaussian_process._gpr', 'GaussianProcessRegressor')
scoring ('sklearn.isotonic', 'IsotonicRegression')
skipping sklearn.isotonic
scoring ('sklearn.kernel_ridge', 'KernelRidge')
scoring ('sklearn.linear_model._bayes', 'ARDRegression')
scoring ('sklearn.linear_model._bayes', 'BayesianRidge')
scoring ('sklearn.linear_model._coordinate_descent', 'ElasticNet')
scoring ('sklearn.linear_model._coordinate_descent', 'ElasticNetCV')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: StackingRegressor.__init__() missing 1 required positional argument: 'estimators'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 21, in train_models
    clf = algorithm()
TypeError: VotingRegressor.__init__() missing 1 required positional argument: 'estimators'
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/isotonic.py", line 351, in fit
    X, y = self._build_y(X, y, sample_weight)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/isotonic.py", line 266, in _build_y
    self._check_input_data_shape(X)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklear

scoring ('sklearn.linear_model._glm.glm', 'GammaRegressor')
skipping sklearn.linear_model._glm.glm
scoring ('sklearn.linear_model._huber', 'HuberRegressor')
scoring ('sklearn.linear_model._least_angle', 'Lars')
scoring ('sklearn.linear_model._least_angle', 'LarsCV')
scoring ('sklearn.linear_model._coordinate_descent', 'Lasso')
scoring ('sklearn.linear_model._coordinate_descent', 'LassoCV')
scoring ('sklearn.linear_model._least_angle', 'LassoLars')
scoring ('sklearn.linear_model._least_angle', 'LassoLarsCV')
scoring ('sklearn.linear_model._least_angle', 'LassoLarsIC')
scoring ('sklearn.linear_model._base', 'LinearRegression')
scoring ('sklearn.linear_model._coordinate_descent', 'MultiTaskElasticNet')
scoring ('sklearn.linear_model._coordinate_descent', 'MultiTaskElasticNetCV')
scoring ('sklearn.linear_model._coordinate_descent', 'MultiTaskLasso')
scoring ('sklearn.linear_model._coordinate_descent', 'MultiTaskLassoCV')


...

scoring ('sklearn.linear_model._ransac', 'RANSACRegressor')
skipping sklearn.linear_model._ransac
scoring ('sklearn.linear_model._ridge', 'Ridge')
scoring ('sklearn.linear_model._ridge', 'RidgeCV')
scoring ('sklearn.linear_model._stochastic_gradient', 'SGDRegressor')
scoring ('sklearn.linear_model._theil_sen', 'TheilSenRegressor')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/linear_model/_ransac.py", line 357, in fit
    raise ValueError(
ValueError: `min_samples` needs to be explicitly set when estimator is not a LinearRegression.


scoring ('sklearn.linear_model._glm.glm', 'TweedieRegressor')
skipping sklearn.linear_model._glm.glm
scoring ('sklearn.multioutput', 'MultiOutputRegressor')
skipping sklearn.multioutput
skipping sklearn.multioutput
scoring ('sklearn.neighbors._regression', 'KNeighborsRegressor')
scoring ('sklearn.neighbors._regression', 'RadiusNeighborsRegressor')
skipping sklearn.neighbors._regression
scoring ('sklearn.neural_network._multilayer_perceptron', 'MLPRegressor')


Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py", line 426, in score
    deviance = base_loss(
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/_loss/loss.py", line 408, in __call__
    self.loss(
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/_loss/loss.py", line 192, in loss
    return self.closs.loss(
  File "sklearn/_loss/_loss.pyx", line 1493, in sklearn._loss._loss.CyHalfTweedieLossIdentity.__pyx_fused_cpdef
TypeError: No matching signature found
Traceback (most recent call last):
  File "/tmp/ipykernel_2987/998152166.py", line 24, in train_models
    score = clf.fit(X=X_train, y=y_train).score(X_test, y_test)
  File "/home/seokmin/miniconda3/envs/research/lib/python3.10/site-packages/

scoring ('sklearn.svm._classes', 'LinearSVR')
scoring ('sklearn.svm._classes', 'NuSVR')
scoring ('sklearn.svm._classes', 'SVR')
scoring ('sklearn.tree._classes', 'DecisionTreeRegressor')
scoring ('sklearn.tree._classes', 'ExtraTreeRegressor')
scoring ('sklearn.compose._target', 'TransformedTargetRegressor')
